In [95]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
import os
from tqdm import tqdm_notebook as tqdm
import time
from lxml import etree
from sklearn.metrics import r2_score
from datetime import timedelta
import numpy as np

In [96]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def pretty_print_result(search_result, fields=None):
    if fields is None:
        fields = []
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')


def get_score(search_result):
    res = []
    for hit in search_result['hits']['hits']:
        res.append((hit["_id"], hit["_score"]))
    res.sort(key = lambda x: x[1], reverse = True)
    return res


class Index:
    def __init__(self, index, settings):
        self.index_name = index
        self.settings = settings
        self.es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360}])
        if self.es.indices.exists(index=index):
            self.es.indices.delete(index=index)
        self.es.indices.create(index=index, body=settings)

    def es_actions_generator(self, path_to_docs):
        for doc_name in tqdm(os.listdir(path_to_docs)):
            with open(f"{path_to_docs}/{doc_name}", "r", encoding="utf-8") as inf:
                doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
                doc = json.load(inf)           
            yield create_es_action(self.index_name, doc_id, doc)


    def add_documents(self, path_to_docs):
        try:
            for ok, result in parallel_bulk(self.es, self.es_actions_generator(path_to_docs), queue_size=4, thread_count=4,
                                        chunk_size=1000):
                  if not ok:
                     print(result)
        except Exception as e: 
            print(e)

    def get_doc_by_id(self, doc_id):
        return self.es.get(index=self.index_name, id=doc_id)['_source']

    def search(self, query, *args):
        return self.es.search(index=self.index_name, body=query, size=20)
        # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20



In [97]:
settings_1 = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            }
        }
    }
}

In [98]:
settings = {
        'mappings': {
            'properties': {
                'text': {
                    'type': 'text',
                    'analyzer': 'russian_complex',
                    'search_analyzer': 'russian_complex'
                }
            }
            
        },
        "settings": {
        "analysis" : {
            "analyzer" : {
                "my_analyzer" : {
                    "tokenizer" : "standard",
                    "filter" : ["lowercase", "russian_snow", "english_snow"]
                },
                'russian_complex': {
                    'char_filter': [
                        'yont'
                    ],
                    'tokenizer': 'word_longer_2',
                    'filter': [
                        'lowercase',
                        'russian_snow'
                    ]
                }
            },
            'char_filter': {
                'yont': {
                    'type': 'mapping',
                    'mappings': [
                        'ё => е'
                    ]
                }
            },
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '[a-zA-Z_0-9\u0400-\u04FF]{2,}',
                    'group': 0
                },
                'white_20': {
                    'type': 'whitespace',
                    'max_token_length': 5
                }
            },
            "filter" : {
                "russian_snow" : {
                    "type" : "snowball",
                    "language" : "Russian"
                },
                "english_snow" : {
                    "type" : "snowball",
                    "language" : "English"
                }
            }
        }
    }
}

In [99]:
index = Index("docs", settings_1)

In [64]:
start = time.time()
index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:04:22.547225


In [65]:
stem_index = Index('stem_docs', settings)

In [66]:
start = time.time()
stem_index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:05:26.785592


In [67]:
stem_index.get_doc_by_id('1000039')

{'text': 'сервисы | о компании | партнерам Описание Рубрикатор Мобильный переводчик SMS-ИНФО - услуга, позволяющая с помощью SMS-запроса оперативно получать информацию различного \nхарактера в текстовом виде. Информация SMS-ИНФО мобильна, доступна 24 часа в сутки на всей территории \nзоны действия сети VELCOM. Ответ на запрос приходит оперативно - в течение нескольких секунд. Для получения информации необходимо отправить SMS-запрос (латиницей или кириллицей) на номер 511. Кроме того, можно сформировать \nтакой SMS-запрос с помощью SIM-карты нового образца с VELCOM-меню. Таблица транслитерации (соответствия \nрусских букв латинским) представлена в разделе Инструкция по SMS-ИНФО. Полный список рубрик SMS-ИНФО можно получить, отправив SMS-запрос HELP на номер 511. Стоимость одного запроса на номер 511 составляет 110 рублей без учета НДС и налога с продаж; для абонентов. Подробнее об услуге SMS-ИНФО читайте в разделе Инструкция по SMS-ИНФО. Внимание! Информацию можно получать и в виде MMS-

In [100]:
class Query:
    def __init__(self, task_id, query, relevant_docs):
        self.task_id = task_id
        self.query = query
        self.relevant_docs = relevant_docs


In [101]:

def json_query(query):
    return {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                }
            ]
        }
    }
}
        


In [102]:
def pagerank_query(query):
    return  {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                },
                {
                    'rank_feature': {
                        'field': 'pagerank',
                        'saturation': {
                            'pivot': 10
                        },
                        'boost': '5.0'
                    }
                }
            ]
        }
    }
}

In [120]:
class Metrics:
    def __init__(self, p, r, r_precision, map_score):
        self.r = r
        self.p = p
        self.r_precision = r_precision
        self.map_score = map_score
    
    def __str__(self):
        return f"r = {self.r}\np = {self.p}\nr_precision = {self.r_precision}\nMAP = {self.map_score}"
    
    __repr__ = __str__


class SearchQualityChecker:
    def __init__(self, queries, index):
        self.queries = queries
        self.index = index
        self.results = {}
        self.metrics = {}
        
    def get_results(self, get_query=json_query):
        r_precision_total = 0
        map_score_total = 0
        r_total = 0
        p_total = 0
        for q in tqdm(self.queries):
            res = self.index.search(get_query(q))
            print(q.task_id)
            pretty_print_result(res)
            scores = get_score(res)
            metric = Metrics(p=self.p(20, q, scores), r=self.r(20, q, scores), r_precision=self.r_precision(q, scores),
                            map_score=self.map_score(q, scores, 20))
            p_total += metric.p
            r_total += metric.r
            r_precision_total += metric.r_precision
            map_score_total += metric.map_score
            self.metrics[q.task_id] = metric
        Q = len(self.queries)
        print(Q)
        return Metrics(p=p_total / Q, r=r_total / Q, r_precision=r_precision_total / Q, map_score=map_score_total / Q)
    
    def r_precision(self, query, search_res_score):
        return self.r(len(query.relevant_docs), query, search_res_score)
    
    def map_score(self, query, search_res_score, n):
        m = 0
        for k in range(1, n):       
            m += self.p(k, query, search_res_score)
        R = len(query.relevant_docs)
        return m / n
    
    def p(self, k, query, search_res_score):
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / k
    
    def r(self, k, query, search_res_score):
        R = len(query.relevant_docs)
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / R if R != 0 else 0 if len(search_res_score) > 0 else 1



In [117]:
def get_relevance():
    res = {}
    xml_tree = etree.parse("data/or_relevant-minus_table.xml")
    root = xml_tree.getroot()
    for task in root.getchildren():
        relevant_docs = set()
        for document in task.getchildren():
            if document.get("relevance") == "vital":
                relevant_docs.add(document.get("id"))
        res[task.get("id")] = relevant_docs
    print(len(res))
    return res

task_id_to_q = {}
def generate_queries_plain_texts():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in root.getchildren():
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    task_id_to_q[task.get("id")] = query_text.text
                    res.append(Query(task.get("id"), query_text.text, relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res

def find_diff_metrics(quality_checker, other_checker, k=20, comp=lambda x : x.map_score):
     res = []
     for task_id in quality_checker.metrics:
        metric = quality_checker.metrics[task_id]
        other_metric = other_checker.metrics[task_id]
        res.append((task_id_to_q[task_id], abs(comp(metric) - comp(other_metric)), metric, other_metric))
     res.sort(reverse=True, key=lambda x: x[1])
     return res

In [112]:
relev = get_relevance()

547


In [113]:
relev['arw30000']

KeyError: 'arw30000'

In [121]:
queries = generate_queries_plain_texts()
#print(queries)
quality_checker = SearchQualityChecker(queries, index)
plain_text_res = quality_checker.get_results()

547
547


arw49633
Total documents: 0
arw49662
Total documents: 0
arw49674
Total documents: 0
arw49692
Total documents: 0
arw49707
Total documents: 0
arw49720
Total documents: 0
arw49723
Total documents: 0
arw49731
Total documents: 0
arw49740
Total documents: 0
arw49745
Total documents: 0
arw49782
Total documents: 0
arw49838
Total documents: 0
arw49843
Total documents: 0
arw49893
Total documents: 0
arw49917
Total documents: 0
arw49924
Total documents: 0
arw49936
Total documents: 0
arw49938
Total documents: 0
arw49944
Total documents: 0
arw49950
Total documents: 0
arw49953
Total documents: 0
arw49958
Total documents: 0
arw49983
Total documents: 0
arw50017
Total documents: 0
arw50040
Total documents: 0
arw50064
Total documents: 0
arw50082
Total documents: 0
arw50104
Total documents: 0
arw50109
Total documents: 0
arw50122
Total documents: 0
arw50191
Total documents: 0
arw50197
Total documents: 0
arw50201
Total documents: 0
arw50227
Total documents: 0
arw50231
Total documents: 0
arw50276
Total docum

arw55770
Total documents: 0
arw55786
Total documents: 0
arw55807
Total documents: 0
arw55811
Total documents: 0
arw55815
Total documents: 0
arw55856
Total documents: 0
arw55859
Total documents: 0
arw55894
Total documents: 0
arw55895
Total documents: 0
arw55963
Total documents: 0
arw55989
Total documents: 0
arw55998
Total documents: 0
arw55999
Total documents: 0
arw56015
Total documents: 0
arw56036
Total documents: 0
arw56043
Total documents: 0
arw56056
Total documents: 0
arw56058
Total documents: 0
arw56068
Total documents: 0
arw56082
Total documents: 0
arw56114
Total documents: 0
arw56131
Total documents: 0
arw56143
Total documents: 0
arw56153
Total documents: 0
arw56155
Total documents: 0
arw56173
Total documents: 0
arw56192
Total documents: 0
arw56202
Total documents: 0
arw56218
Total documents: 0
arw56232
Total documents: 0
arw56247
Total documents: 0
arw56345
Total documents: 0
arw56405
Total documents: 0
arw56407
Total documents: 0
arw56409
Total documents: 0
arw56439
Total docum

In [122]:
plain_text_res

r = 0.09506398537477148
p = 0.0
r_precision = 0.09506398537477148
MAP = 0.0

In [115]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")
black_list = ["°", "№", "©", "...", "//", "://", "</", "\">", "=\"", "=\'", "\r", "\n", "\t"]
stem = Mystem()

def lemmatize(text):
    words = nltk.word_tokenize(text.lower())
    tokens = []
    for word in words:
        tokens.extend(stem.lemmatize(word))
    tokens = [token for token in tokens if token != " " and token.strip() not in punctuation \
              and token not in russian_stopwords and token not in english_stopwords \
              and token not in black_list \
              and token.find("\r") == -1 \
              and token.find("\n") == -1 \
              and token.find("\t") == -1 \
              and not (token.isdigit() and len(token) == 1)]
    return ' '.join(tokens)

def generate_queries_lemmas():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in tqdm(root.getchildren()):
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), lemmatize(query_text.text), relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res



In [116]:
queries_lemmas = generate_queries_lemmas()

547



547


In [78]:
queries_lemmas[8].query

'инвесткапиталбанк'

In [79]:
lemma_index = Index("lemma_docs", settings_1)

start = time.time()
lemma_index.add_documents("data/json_filtered_tokens_texts")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:03:15.451904


In [123]:
lemma_quality_checker = SearchQualityChecker(queries_lemmas, lemma_index)
lemma_res = lemma_quality_checker.get_results()

arw49633
Total documents: 5080
Doc 914238, score is 13.799336
Doc 1393503, score is 13.701229
Doc 1407642, score is 13.604507
Doc 567839, score is 13.571222
Doc 1385740, score is 13.571222
Doc 1504166, score is 12.54204
Doc 1016244, score is 12.132923
Doc 993272, score is 11.921361
Doc 990390, score is 11.921361
Doc 359461, score is 11.921361
Doc 360857, score is 11.359218
Doc 495806, score is 10.947564
Doc 854005, score is 10.142758
Doc 691503, score is 9.414591
Doc 1340878, score is 9.019914
Doc 1319901, score is 7.9332757
Doc 35586, score is 7.8946466
Doc 114021, score is 7.8873105
Doc 599962, score is 7.8450575
Doc 863522, score is 7.8382044
arw49662
Total documents: 0
arw49674
Total documents: 10000
Doc 1503640, score is 22.287449
Doc 674640, score is 22.287449
Doc 876069, score is 21.912388
Doc 49202, score is 20.201603
Doc 575107, score is 19.590178
Doc 1109272, score is 19.590178
Doc 664883, score is 19.063917
Doc 74265, score is 18.712236
Doc 284872, score is 18.100986
Doc 131

arw49944
Total documents: 10000
Doc 7317, score is 20.949793
Doc 942354, score is 20.436937
Doc 624744, score is 19.993721
Doc 1173677, score is 19.758795
Doc 1480002, score is 19.615366
Doc 362460, score is 19.473494
Doc 1339324, score is 19.33974
Doc 1309622, score is 19.226002
Doc 1369472, score is 19.144844
Doc 1296174, score is 19.084396
Doc 1339827, score is 19.04328
Doc 721485, score is 19.01279
Doc 811305, score is 18.912838
Doc 1295722, score is 18.862757
Doc 1479613, score is 18.62207
Doc 1401652, score is 18.451096
Doc 1314801, score is 18.273771
Doc 857813, score is 18.250877
Doc 378916, score is 17.994081
Doc 696409, score is 17.863583
arw49950
Total documents: 10000
Doc 866689, score is 24.263437
Doc 1299659, score is 24.263437
Doc 489340, score is 24.096928
Doc 494355, score is 23.283482
Doc 1012870, score is 22.982218
Doc 269146, score is 22.97311
Doc 586658, score is 22.898985
Doc 586432, score is 22.726059
Doc 583987, score is 22.726059
Doc 448137, score is 22.604153


arw50276
Total documents: 10000
Doc 4053, score is 22.949055
Doc 675632, score is 19.641657
Doc 878847, score is 19.247711
Doc 771209, score is 19.166359
Doc 422593, score is 18.929113
Doc 26891, score is 18.830061
Doc 47141, score is 18.696009
Doc 649450, score is 18.668884
Doc 1449392, score is 18.485468
Doc 815863, score is 18.419012
Doc 417668, score is 18.243214
Doc 1365208, score is 18.068588
Doc 679077, score is 17.922443
Doc 988276, score is 17.8506
Doc 280888, score is 17.714481
Doc 968919, score is 17.714481
Doc 41888, score is 17.666801
Doc 1253795, score is 17.597158
Doc 474809, score is 17.428158
Doc 522355, score is 17.425522
arw50280
Total documents: 9877
Doc 858190, score is 19.791473
Doc 969597, score is 17.541035
Doc 251914, score is 17.37619
Doc 983109, score is 17.069805
Doc 1136235, score is 16.337282
Doc 872253, score is 16.337282
Doc 1378648, score is 15.773499
Doc 47426, score is 15.609777
Doc 878497, score is 15.38065
Doc 880747, score is 14.811039
Doc 1435138,

arw50554
Total documents: 10000
Doc 345185, score is 24.099396
Doc 262690, score is 24.025612
Doc 345151, score is 24.025612
Doc 1014941, score is 23.87983
Doc 975894, score is 23.87983
Doc 811298, score is 23.512405
Doc 942482, score is 22.724968
Doc 1013100, score is 22.278471
Doc 976116, score is 22.197683
Doc 975879, score is 22.185574
Doc 1099902, score is 21.845137
Doc 262686, score is 21.760603
Doc 345180, score is 21.760603
Doc 1100437, score is 21.694735
Doc 29035, score is 21.60046
Doc 975925, score is 21.44722
Doc 583410, score is 21.39835
Doc 117519, score is 21.18834
Doc 1390461, score is 21.174843
Doc 345130, score is 21.060555
arw50570
Total documents: 1035
Doc 942328, score is 38.501312
Doc 117451, score is 36.326347
Doc 565497, score is 21.746073
Doc 870682, score is 20.051968
Doc 366438, score is 16.123363
Doc 117450, score is 15.802971
Doc 1357039, score is 14.343457
Doc 14701, score is 14.127384
Doc 1457840, score is 14.074379
Doc 271110, score is 14.073408
Doc 1415

arw50882
Total documents: 10000
Doc 555823, score is 16.200846
Doc 1461224, score is 16.167263
Doc 1369813, score is 16.167263
Doc 721769, score is 16.167263
Doc 55205, score is 16.167263
Doc 582597, score is 15.59793
Doc 629298, score is 15.402403
Doc 1463068, score is 15.402403
Doc 986592, score is 15.402403
Doc 723113, score is 15.402403
Doc 500408, score is 15.262093
Doc 1132232, score is 15.248726
Doc 495990, score is 15.1806755
Doc 264600, score is 14.902269
Doc 497097, score is 14.851916
Doc 1017338, score is 14.669085
Doc 550647, score is 14.657657
Doc 1295896, score is 14.651387
Doc 1370494, score is 14.547202
Doc 825538, score is 14.448936
arw50885
Total documents: 10000
Doc 48644, score is 15.000532
Doc 1215048, score is 14.944739
Doc 356431, score is 14.935413
Doc 701450, score is 14.935413
Doc 860954, score is 14.926313
Doc 1249603, score is 14.907803
Doc 1029799, score is 14.907803
Doc 355577, score is 14.796501
Doc 539471, score is 14.796501
Doc 1353433, score is 14.7336

...

arw57780
Total documents: 10000
Doc 66224, score is 14.772831
Doc 1136428, score is 14.086187
Doc 442500, score is 13.733908
Doc 792566, score is 13.610376
Doc 37738, score is 13.584221
Doc 671326, score is 13.584221
Doc 756, score is 13.500149
Doc 1256, score is 13.500149
Doc 1064754, score is 13.441223
Doc 1283696, score is 13.434716
Doc 1472200, score is 13.399244
Doc 530247, score is 13.389842
Doc 872815, score is 13.3070545
Doc 37737, score is 13.301415
Doc 901270, score is 13.282023
Doc 53480, score is 13.246046
Doc 986928, score is 13.246046
Doc 1196346, score is 13.242689
Doc 1295306, score is 13.225278
Doc 495229, score is 13.167212
arw57796
Total documents: 545
Doc 288197, score is 11.972996
Doc 826937, score is 11.900883
Doc 941027, score is 11.900883
Doc 1211020, score is 11.89547
Doc 1486360, score is 11.830142
Doc 891675, score is 11.699818
Doc 869844, score is 11.635059
Doc 1300219, score is 11.635059
Doc 449962, score is 11.611083
Doc 1211150, score is 11.258378
Doc 821

Doc 820667, score is 12.002942
Doc 319083, score is 11.688693
Doc 69947, score is 11.333111
Doc 295338, score is 11.271469
arw58075
Total documents: 3
Doc 440227, score is 14.791274
Doc 1502239, score is 8.960033
Doc 794459, score is 8.960033
arw58091
Total documents: 6854
Doc 465440, score is 16.277695
Doc 1369844, score is 16.207653
Doc 1215858, score is 15.603117
Doc 489486, score is 14.764254
Doc 579824, score is 13.774291
Doc 1311702, score is 13.76248
Doc 606020, score is 13.263706
Doc 1304366, score is 13.187832
Doc 1174799, score is 12.720526
Doc 789473, score is 12.718945
Doc 1303808, score is 12.553311
Doc 1522815, score is 12.553311
Doc 1383860, score is 12.316208
Doc 1062867, score is 12.316208
Doc 1438138, score is 12.21697
Doc 1062876, score is 12.114094
Doc 1307866, score is 12.041233
Doc 1508, score is 12.041233
Doc 1472142, score is 12.041233
Doc 1483681, score is 12.041233
arw58105
Total documents: 10000
Doc 254958, score is 14.551487
Doc 306674, score is 14.495684
Do

Doc 1184393, score is 10.463611
Doc 1212955, score is 10.144438
Doc 290692, score is 9.941316
Doc 1413460, score is 9.715718
Doc 1185524, score is 9.641542
Doc 1375331, score is 9.580658
Doc 499930, score is 9.493524
Doc 373197, score is 9.479729
Doc 10556, score is 9.404857
Doc 1102138, score is 9.366734
Doc 837585, score is 9.3624525
Doc 1438155, score is 9.308499
Doc 1342135, score is 9.255008
Doc 1038512, score is 9.241669
Doc 1429272, score is 9.182791
Doc 269044, score is 9.172964
Doc 291730, score is 9.1679325
Doc 1343238, score is 9.1679325
Doc 1008617, score is 9.126504
Doc 1317114, score is 9.12624
arw58367
Total documents: 10000
Doc 289175, score is 23.005093
Doc 1207656, score is 22.657616
Doc 565677, score is 21.709888
Doc 587740, score is 21.450203
Doc 65401, score is 21.422377
Doc 345295, score is 21.272724
Doc 581596, score is 21.26323
Doc 345398, score is 21.056969
Doc 496746, score is 20.31652
Doc 1013397, score is 20.305555
Doc 97296, score is 20.131762
Doc 265994, s

arw58635
Total documents: 10000
Doc 1175993, score is 20.202055
Doc 1432566, score is 19.585688
Doc 13664, score is 19.400328
Doc 1338748, score is 19.320623
Doc 374549, score is 18.909096
Doc 1192644, score is 18.682201
Doc 251909, score is 18.611181
Doc 884083, score is 18.14953
Doc 670299, score is 18.115309
Doc 681982, score is 17.99147
Doc 112544, score is 17.98618
Doc 1027149, score is 17.974731
Doc 884024, score is 17.961994
Doc 324434, score is 17.940313
Doc 1036637, score is 17.819166
Doc 1102292, score is 17.707811
Doc 930956, score is 17.658352
Doc 628778, score is 17.322922
Doc 618395, score is 17.14383
Doc 1180989, score is 17.129427
arw58642
Total documents: 10000
Doc 293718, score is 16.622305
Doc 1016103, score is 16.19177
Doc 1491107, score is 15.559959
Doc 809588, score is 15.497698
Doc 688000, score is 15.3197775
Doc 938863, score is 15.302586
Doc 1037761, score is 15.271928
Doc 63780, score is 15.113415
Doc 884211, score is 15.037497
Doc 443660, score is 15.009531
D

arw58958
Total documents: 6653
Doc 534475, score is 32.570084
Doc 690561, score is 32.206234
Doc 614546, score is 31.906248
Doc 1275812, score is 29.957657
Doc 1147078, score is 29.417034
Doc 621531, score is 29.090626
Doc 620719, score is 28.861893
Doc 386081, score is 28.741215
Doc 1470406, score is 28.640118
Doc 1471385, score is 28.582157
Doc 1033730, score is 28.57412
Doc 283819, score is 28.495167
Doc 459477, score is 28.284939
Doc 1007008, score is 28.121979
Doc 688660, score is 27.73383
Doc 532539, score is 27.621464
Doc 601244, score is 25.93834
Doc 76461, score is 25.93834
Doc 601524, score is 25.93834
Doc 862312, score is 25.93834
arw58970
Total documents: 10000
Doc 16942, score is 13.816998
Doc 942385, score is 13.163893
Doc 257487, score is 13.000159
Doc 1401472, score is 12.903826
Doc 626159, score is 12.842594
Doc 1421173, score is 12.696351
Doc 1402179, score is 12.518756
Doc 388790, score is 12.4870615
Doc 910530, score is 12.426571
Doc 1262228, score is 12.303615
Doc 

In [81]:
lemma_res

r = 0.2535327962534067
p = 0.3577696526508227
r_precision = 0.2121663681252442
MAP = 0.38583429879378817

In [124]:
find_diff_metrics(quality_checker, lemma_quality_checker)

[('борьба с терроризмом', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.29411764705882354
  p = 1.0
  r_precision = 0.29411764705882354
  MAP = 0.95), ('форд фокус', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.4
  p = 1.0
  r_precision = 0.4
  MAP = 0.95), ('Вакансии медицина фармация', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.37037037037037035
  p = 1.0
  r_precision = 0.37037037037037035
  MAP = 0.95), ('гипоксия', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.3389830508474576
  p = 1.0
  r_precision = 0.3389830508474576
  MAP = 0.95), ('Фитнес клуб Палестра', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.18867924528301888
  p = 1.0
  r_precision = 0.18867924528301888
  MAP = 0.95), ('Спа центры', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.2898550724637681
  p = 1.0
  r_precision = 0.2898550724637681
  MAP = 0.95), ('газпромбанк', 0.95, r = 0.0
  p = 0.0
  r_precision = 0.0

In [119]:
id_to_pagerank = {}
with open('res/pagerank.txt','r') as f:
    for line in f:
        docId, docURL, rank = line.split()
        id_to_pagerank[int(docId)] = float(rank)

In [ ]:
len(id_to_pagerank)


In [ ]:
 for doc_name in tqdm(os.listdir("data/json_filtered_tokens_texts")):
        with open(f"data/json_filtered_tokens_texts/{doc_name}", "r+", encoding="utf-8") as inf:
            doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
            doc = json.load(inf)
            try:
                doc["pagerank"] = id_to_pagerank.get(doc_id)
            except:
                pass
            inf.seek(0)        # <--- should reset file position to the beginning.
            json.dump(doc, inf, indent=4, ensure_ascii=False)
            inf.truncate()

In [ ]:
settings_with_pagerank = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "pagerank": {
                "type": "rank_feature"
            }
        }
    }
}

In [ ]:
pr_index = Index("pagerank_index", settings_with_pagerank)

In [ ]:
pr_index.add_documents("data/json_filtered_tokens_texts")

In [ ]:
pr_quality_checker = SearchQualityChecker(queries_lemmas, pr_index)
pr_res = pr_quality_checker.get_results(pagerank_query)

In [ ]:
pr_res

In [ ]:
pr_index.get_doc_by_id(1000039)

In [ ]:
settings_titles = {
        'mappings': {
            'properties': {
                'content': {
                    'type': 'text',
                },
                'title': {
                    'type': 'text'
                }
            }
            
        }
    }